# CBTN learning lab for multi-modal imaging
#### October 10, 2024

The Flywheel Python SDK is a powerful resource for data management and processing on Flywheel. This notebook illustrates how to run pre-processing and automated tumor segmentation, to implement the same steps we completed in the web browser interface.

For documentation and additional examples, please see:
- [Flywheel SDK main documentation](https://flywheel-io.gitlab.io/product/backend/sdk/tags/18.1.0/python/index.html#)
- [CHOP-D3b repository of SDK examples](https://github.com/d3b-center/d3b-tiru-flywheel-code-repo)

In [9]:
import flywheel
from datetime import datetime

# access the Flywheel client
your_api_key = 'enter-your-api-key'
fw = flywheel.Client(your_api_key)

### Run pre-processing using the Flywheel SDK (single session)

In [10]:
# define the project to run the gear in
project_group = 'cbtn-summit'
project_name = 'CBTN_raw_data'
# load the project container using the find function
project = fw.projects.find_first(f'label={project_name}')

# define the subject and session labels to process
sub_id = 'C1173420'
ses_id = '697d_B_brain_09h58m'

# define the acquisition labels to use as gear inputs
acq_map = {'T1':    '09 - t1 3D cor mprage',
           'T1CE':  '14 - t1 3D ax mprage (stealth)',
           'T2':    '10 - t2_tse_tra',
           'FLAIR': '04 - t2_tirm_fs_tra_dark-fluid_320'
            }

## Initialize the gear
gear_name = 'captk-brats-pipeline'
gear2run = fw.lookup(f'gears/{gear_name}')

## load the file containers that we want to input into the gear
session = fw.lookup(f'{project_group}/{project_name}/{sub_id}/{ses_id}')
gear_inputs = {'FLAIR':[], \
            'T1':[], \
            'T1CE':[], \
            'T2':[]
            }

for acquisition in session.acquisitions.iter():
    if any(acquisition.label in value for value in acq_map.values()): # if this acquisition matches one of the 4 in our dictionary
        # find the nifti file container within this acquisition
        im_type = [key for key, value in acq_map.items() if acquisition.label in value][0]
        for file_obj in acquisition.files:
            if file_obj.type == 'nifti':
                gear_inputs[im_type] = file_obj

# queue the gear to run on the Flywheel instance
time_fmt = '%m/%d/%Y %H:%M:%S'
analysis_label = f'{gear_name} {datetime.now().strftime(time_fmt)}'
job_id = gear2run.run(analysis_label = analysis_label, \
                        inputs = gear_inputs, \
                        destination = session )


### Run automated tumor segmentation gear on pre-processed images (multi-session)

Now we'll assume the BraTS pipeline has been run for all selected sessions in our experiment and there's a corresponding analysis container at the session-level.

Next we'll run gears to generate a brain mask and tumor segmentation based on pre-trained deep learning models.

In [ ]:
# we're going to take the pre-processed images from the BraTS analysis containers
# and input the files into the automated tumor segmentation gear
source_gear = 'captk-brats-pipeline'
destination_gear = 'd3b-ped-auto-seg-nnunet'

## Initialize gear
gear2run = fw.lookup(f'gears/{destination_gear}')

# loop through all session containers in the project
for session in project.sessions.iter():
    gear_inputs = {'t1':[],
                    't1ce':[],
                    't2':[],
                    'flair':[],
                    }
    # Make sure we have all our analysis since we got the session through an iterator, and not "fw.get()'
    ses = ses.reload()
    # load the analysis containers attached to this session
    analyses = ses.analyses
    # If there are no analyses containers, we know that this gear was not run. 
    if len(analyses) > 0:
        # Check to see if any were generated by our pre-processing gear
        matches = [asys for asys in analyses if asys.gear_info.get('name') == source_gear]
        # make sure these were successful runs
        matches = [match for match in matches if match.job.state in ['complete']]
        if len(matches) > 0:
            match = matches[0] # grab first successful match
            # define the gear inputs based on the known file naming of the pre-processed images
            for output_file in match.files:
                if 'T1_to_SRI' in output_file.name:
                    gear_inputs['t1'] = output_file
                elif 'T1CE_to_SRI' in output_file.name:
                    gear_inputs['t1ce'] = output_file
                elif 'T2_to_SRI' in output_file.name:
                    gear_inputs['t2'] = output_file
                elif 'FL_to_SRI' in output_file.name:
                    gear_inputs['flair'] = output_file
        # queue gear to run
        time_fmt = '%m/%d/%Y %H:%M:%S'
        analysis_label = f'{gear_name} {datetime.now().strftime(time_fmt)}'
        job_id = gear2run.run(analysis_label = analysis_label, \
                                    inputs = gear_inputs, \
                                    destination = session )
